In [31]:

# Import libraries

import simpy
import random
import statistics

# Set Simulation Global Variables
PASSENGER_INTERVAL = 5
NUM_FLOORS = 8
NUM_ELEVATORS = 4
MAX_CAPACITY = 4
MINUTES_IN_DAY = 24 * 60 # Total simulation time in minutes
SIM_TIME = MINUTES_IN_DAY # Total simulation time in minutes
CHECK_INTERVAL = 1

In [32]:
def get_arrival_rate(current_time_minutes, floor):
    """
    returns the arrival rate  based on time of day and floor.
    
    arrival patterns:
    Morning (7:30-8:30 AM): High ground floor arrivals 
    Lunch (12:00-1:00 PM): High arrivals from all floors 
    Evening (5:00-6:00 PM): High arrivals from upper floors (people leaving work)
    Night/Off-peak: Low arrivals from all floors 
    """
    
    # convert time to minutes 
    time_of_day = current_time_minutes % 1440
    
    # hard coded arrival rates 
    high_rate = 0.5      # Peak traffic
    medium_rate = 0.2    # Moderate traffic  
    low_rate = 0.05      # Off-peak traffic
    
    # time periods
    morning_start, morning_end = 450, 510      # 7:30-8:30 AM
    lunch_start, lunch_end = 720, 780          # 12:00-1:00 PM  
    evening_start, evening_end = 1020, 1080   # 5:00-6:00 PM
    
    # determine time period and apply floor-specific rates by checking to see which band the current time falls into
    if morning_start <= time_of_day <= morning_end:
        
        if floor == 0:  
            return high_rate
        else:
            return low_rate
            
    elif lunch_start <= time_of_day <= lunch_end:
        
        if floor == 0:  
            return low_rate  
        else:
            return high_rate  
            
    elif evening_start <= time_of_day <= evening_end:
        
        if floor == 0:  
            return low_rate
        else:
            return high_rate
            
    else:  
        return medium_rate
    

In [33]:
def get_time_period(current_time_minutes):
    
    time_of_day = current_time_minutes % 1440
    
    if 450 <= time_of_day <= 510:
        return "morning"
    elif 720 <= time_of_day <= 780:
        return "lunch"
    elif 1020 <= time_of_day <= 1080:
        return "evening"
    else:
        return "off_peak"

In [34]:
def generate_destination(pickup_floor, time_period, num_floors):
    """
    generate destinations based on pickup floor and time of day
    """

    if time_period == "morning" and pickup_floor == 0:
        # Morning arrivals at ground floor typically go up
        return random.randint(1, num_floors - 1)
    
    elif time_period in ["lunch", "evening"] and pickup_floor > 0:

        # Lunch/evening departures from upper floors typically go to ground
        if random.random() < 0.8:  # 80% chance to go to ground floor
            return 0
        
        else:
            # 20% chance to go to another floor
            destination = random.randint(0, num_floors - 1)
            while destination == pickup_floor:
                destination = random.randint(0, num_floors - 1)
            return destination
    else:
        # Random destination for other cases
        destination = random.randint(0, num_floors - 1)
        while destination == pickup_floor:
            destination = random.randint(0, num_floors - 1)
        return destination


In [35]:
class Passenger:
    def __init__(self, pid, pickup, destination, start_time):
        self.id = pid
        self.pickup = pickup
        self.destination = destination
        self.start_time = start_time  # Track when passenger enters system
    
    def __repr__(self, color = '\033[97m'):         # changes representation of passenger to be easier to read
        return f"{color}P{self.id}({self.pickup} --> {self.destination})"

I want to change passenger_generator a little bit. 
This now has all passengers arriving at a rate of 1 / lambda. 
    The issue is floor 1 will have many more people arriving than floor 2, 3, ....
    I think it would be reasonable to have X people arriving at floor 1 and then
        floor 2 and 3 would have X/2 or if we have 5 floors each would have X / 4. 
            - Cree

In [36]:
class Dispatcher:
    def __init__(self, elevators):
        self.elevators = elevators

    def assign_passenger(self, passenger):
        best_elevator = min(
            self.elevators, key = lambda x: abs(x.floor - passenger.pickup)
        )

        best_elevator.request_pickup(passenger)

In [37]:

def passenger_generator(env, dispatcher, num_floors=NUM_FLOORS, check_interval=CHECK_INTERVAL):
    """
    Generate passengers using Bernoulli trials.
    
    How it works:
    1. Every 'check_interval' minutes, check each floor
    2. For each floor, get the arrival rate (passengers/minute)
    3. Convert rate to probability: P(arrival) = rate × interval
    4. Use random() < probability to decide if passenger arrives
    5. If yes, create passenger with realistic destination
    """
    pid = 1
    
    while True:
        current_time_str = time_to_string(env.now)
        time_period = get_time_period(env.now)
        
        # Debug: Print current rates (optional - remove in production)
        if env.now % 60 == 0:  # Print every hour
            print(f"[{current_time_str}] Time period: {time_period}")
            for floor in range(num_floors):
                rate = get_arrival_rate(env.now, floor)
                prob = rate * check_interval
                print(f"  Floor {floor}: rate={rate:.3f}/min, prob={prob:.3f}")
        
        # Check each floor for potential passenger arrivals
        for floor in range(num_floors):
            arrival_rate = get_arrival_rate(env.now, floor)
            
            # Convert rate to probability for this time interval
            # For small intervals: P(arrival in interval) ≈ rate × interval
            arrival_probability = arrival_rate * check_interval
            
            # Bernoulli trial: does a passenger arrive?
            if random.random() < arrival_probability:
                # Generate realistic destination
                destination = generate_destination(floor, time_period, num_floors)
                
                # Create passenger
                start_time = env.now
                passenger = Passenger(pid, floor, destination, start_time)
                
                # Assign to dispatcher
                dispatcher.assign_passenger(passenger)
                
                # Log passenger creation
                print(f"[{current_time_str}] Passenger {pid} appears at floor {floor}, wants to go to {destination} (Period: {time_period})")
                
                pid += 1
        
        # Wait for next check
        yield env.timeout(check_interval)


In [38]:
def time_to_string(minutes):
    """Convert minutes from midnight to HH:MM format"""
    hours = int(minutes // 60) % 24
    mins = int(minutes % 60)
    return f"{hours:02d}:{mins:02d}"

Elevator currently has a log, pickup_requests, and a run function that onboards and offboards people. It, however, doesn't move up or down floors ¯\_(ツ)_/¯

To start, I think we should go with a super simple:
    if self.floor < num_floors:
        floor += 1
    else: floor -= 1
    that way it goes all the way up and then goes all the way down. picking people up and dropping them off as it goes. 

In [39]:
class Elevator:
    def __init__(self, env, num_floors, max_capacity, strategy = "nearest", eid=0):
        self.env = env
        self.id = eid
        self.floor = 0          # Start at ground floor (0)
        self.direction = 'up'   # up or down
        self.num_floors = num_floors # total
        self.max_capacity = max_capacity
        self.pickup_requests = {floor: [] for floor in range(num_floors)}
        self.onboard = []
        self.strategy = strategy
        self.stats = {
            'total_passengers': 0,
            'total_wait_time': 0,
            'total_travel_time': 0
        }

    def log(self, message):
        current_time = time_to_string(self.env.now)
        print(f"    [{current_time}] Elevator {self.id} at floor: {self.floor} | {message}")

    def request_pickup(self, passenger):
        self.pickup_requests[passenger.pickup].append(passenger)
        self.log(f"Passenger {passenger.id} requests a pickup at floor {passenger.pickup}")

    def find_longest_waiting_request(self):
        longest_wait_time = -1
        target_floor = None
        for floor, passengers in self.pickup_requests.items():
            for p in passengers:
                wait_time = self.env.now - p.start_time
                if wait_time > longest_wait_time:
                    longest_wait_time = wait_time
                    target_floor = floor
        return target_floor

    def run_elevator(self):
        while True:
            # 1) Off-load passengers at current floor
            offboarding = [p for p in self.onboard if p.destination == self.floor]
            for p in offboarding:
                self.onboard.remove(p)
                travel_time = self.env.now - p.start_time
                self.stats['total_travel_time'] += travel_time
                self.stats['total_passengers'] += 1
                self.log(f"Dropped off {p} (travel time: {travel_time:.1f} min)")

            # 2) On-board passengers at current floor
            boarding = self.pickup_requests[self.floor]
            for p in boarding:
                if len(self.onboard) < MAX_CAPACITY:
                    self.onboard.append(p)
                    wait_time = self.env.now - p.start_time
                    self.stats['total_wait_time'] += wait_time
                    self.log(f"Picked up {p} (wait time: {wait_time:.1f} min)")
                else:
                    self.log(f"Elevator full, {p} must wait")
            
            self.pickup_requests[self.floor].clear()

            if not any(self.pickup_requests.values()) and not self.onboard:
                #self.log("idle")
                yield self.env.timeout(1)
                continue


            # 3) Move logic based on strategy
            if self.strategy == "longest_wait":
                target_floor = self.find_longest_waiting_request()
    
                if target_floor is not None and target_floor != self.floor:
                    if target_floor > self.floor:
                        self.direction = 'up'
                        self.floor += 1
                    elif target_floor < self.floor:
                        self.direction = 'down'
                        self.floor -= 1
                        self.log(f"Moving {self.direction} toward floor {target_floor}")
                else:
                # No passengers waiting → continue in current direction
                    self.log("No pickups pending, continuing current direction")
                    if self.direction == 'up':
                        if self.floor < self.num_floors - 1:
                            self.floor += 1
                        else:
                            self.direction = 'down'
                            self.floor -= 1
                    else:
                        if self.floor > 0:
                            self.floor -= 1
                        else:
                            self.direction = 'up'
                            self.floor += 1
            else:
            # Default nearest (up/down sweep)
                if self.direction == 'up':
                    if self.floor < self.num_floors - 1:
                        self.floor += 1
                    else:
                        self.direction = 'down'
                        self.floor -= 1
                else:
                    if self.floor > 0:
                        self.floor -= 1
                    else:
                        self.direction = 'up'
                        self.floor += 1
            self.log(f"Moving {self.direction} to floor {self.floor}")


# 4) Wait for travel time (e.g. 1 time-unit per floor)
            yield self.env.timeout(.5)

In [40]:
class Elevator_Controller:
    def __init__(self, env, NUM_ELEVATORS, NUM_FLOORS):
        self.env = env
        self.elevators = [Elevator(env, NUM_FLOORS, max_capacity=MAX_CAPACITY, eid=i) for i in range(NUM_ELEVATORS)]
        self.dispatcher = Dispatcher(self.elevators)

    def start(self):
        for elevator in self.elevators:
            self.env.process(elevator.run_elevator())

In [41]:
def print_simulation_stats(elevator):
    """Print simulation statistics"""
    print("\n" + "="*50)
    print("SIMULATION STATISTICS")
    print("="*50)
    print(f"Total passengers served: {elevator.stats['total_passengers']}")
    if elevator.stats['total_passengers'] > 0:
        avg_wait_time = elevator.stats['total_wait_time'] / elevator.stats['total_passengers']
        avg_travel_time = elevator.stats['total_travel_time'] / elevator.stats['total_passengers']
        print(f"Average wait time: {avg_wait_time:.1f} minutes")
        print(f"Average travel time: {avg_travel_time:.1f} minutes")
    print(f"Simulation duration: {SIM_TIME} minutes ({SIM_TIME/60:.1f} hours)")
    print(f"total passengers {[elevator.stats['total_passengers']]}")
    print("="*50)
    print(sum(ele.stats['total_passengers'] for ele in controller.elevators))

In [42]:
# Run the simulation
if __name__ == "__main__":
    random.seed(23)
    env = simpy.Environment()
    elevator = Elevator(env, num_floors=NUM_FLOORS, max_capacity=MAX_CAPACITY, strategy="longest_wait")  # or "nearest"
    controller = Elevator_Controller(env, NUM_ELEVATORS, NUM_FLOORS)
    controller.start()
    env.process(passenger_generator(env, controller.dispatcher))
    
    # Start both processes
    # env.process(passenger_generator(env, elevator))
    # env.process(elevator.run_elevator())
    
    print(f"Starting 24-hour elevator simulation...")
    print(f"Simulation will run for {SIM_TIME} minutes ({SIM_TIME/60:.1f} hours)")
    print("-" * 50)
    
    env.run(until=SIM_TIME)
    
    print_simulation_stats(elevator) 

Starting 24-hour elevator simulation...
Simulation will run for 1440 minutes (24.0 hours)
--------------------------------------------------
[00:00] Time period: off_peak
  Floor 0: rate=0.200/min, prob=0.200
  Floor 1: rate=0.200/min, prob=0.200
  Floor 2: rate=0.200/min, prob=0.200
  Floor 3: rate=0.200/min, prob=0.200
  Floor 4: rate=0.200/min, prob=0.200
  Floor 5: rate=0.200/min, prob=0.200
  Floor 6: rate=0.200/min, prob=0.200
  Floor 7: rate=0.200/min, prob=0.200
    [00:00] Elevator 0 at floor: 0 | Passenger 1 requests a pickup at floor 3
[00:00] Passenger 1 appears at floor 3, wants to go to 4 (Period: off_peak)
    [00:00] Elevator 0 at floor: 0 | Passenger 2 requests a pickup at floor 6
[00:00] Passenger 2 appears at floor 6, wants to go to 3 (Period: off_peak)
    [00:01] Elevator 0 at floor: 1 | Moving up to floor 1
    [00:01] Elevator 1 at floor: 0 | Passenger 3 requests a pickup at floor 0
[00:01] Passenger 3 appears at floor 0, wants to go to 7 (Period: off_peak)
    [